# 🚀 Reto de Ingeniería de Datos: Optimización y Visualización (Módulo 1)
**Diplomado en Estrategias de Datos - USTA**  

- **Estudiante:** _[Janis Rodriguez]_  
- **Fecha:** 2026-01-04  

**Descripción:** En este cuaderno aplico técnicas de *downcasting*, vectorización y visualización avanzada sobre el dataset **NYC Taxi Trip Duration** (~1.4M registros).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from fpdf import FPDF

# Configuración: gráficos nítidos y estilo limpio
%config InlineBackend.figure_format = 'retina'
sns.set_theme(style="whitegrid")

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 200)


## 1) Carga y diagnóstico de memoria (El “Antes”)

Cargamos el dataset **sin optimizar** y medimos consumo de memoria real con `memory_usage='deep'`.  
> Nota: **NO subas `train.csv` a GitHub** (pesa >100MB). Debe quedar solo local.


In [ ]:
def find_train_csv():
    candidates = [
        Path("train.csv"),
        Path("data/train.csv"),
        Path("data/raw/train.csv"),
    ]
    for p in candidates:
        if p.exists():
            return p
    raise FileNotFoundError(f"No encontré train.csv en: {candidates}")

train_path = find_train_csv()
train_path


PosixPath('data/raw/train.csv')

In [ ]:
df_raw = pd.read_csv(train_path)
df_raw.shape


In [ ]:
# Diagnóstico de memoria real (deep)
df_raw.info(memory_usage="deep")


In [ ]:
def memory_mb(df: pd.DataFrame) -> float:
    return df.memory_usage(deep=True).sum() / (1024**2)

mem_before = memory_mb(df_raw)
mem_before


## 2) Fase 1: Optimización (Pandas Pro)

Objetivo: **reducir al menos 50%** el consumo de memoria **sin perder información**.

Técnicas:
- Downcasting numérico (`int64 → int8/int16`, `float64 → float32` cuando aplica)
- Conversión de `object` repetitivo a `category`
- Parseo de fechas (datetime) de manera controlada


In [ ]:
def optimize_types(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Parseo de fechas (en el dataset suelen existir pickup_datetime y dropoff_datetime)
    for c in ["pickup_datetime", "dropoff_datetime"]:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")

    # 2) Downcasting numérico
    int_cols = df.select_dtypes(include=["int64", "int32", "int16"]).columns
    for c in int_cols:
        df[c] = pd.to_numeric(df[c], downcast="integer")

    float_cols = df.select_dtypes(include=["float64"]).columns
    for c in float_cols:
        df[c] = pd.to_numeric(df[c], downcast="float")

    # 3) Objects -> category (solo si vale la pena: baja cardinalidad relativa)
    obj_cols = df.select_dtypes(include=["object"]).columns
    n = len(df)
    for c in obj_cols:
        nunique = df[c].nunique(dropna=False)
        # Heurística: si la columna tiene pocos valores únicos
        if nunique <= min(50000, max(50, int(0.5 * n))):
            df[c] = df[c].astype("category")

    return df

df_opt = optimize_types(df_raw)
df_opt.shape


In [ ]:
df_opt.info(memory_usage="deep")


In [ ]:
mem_after = memory_mb(df_opt)
reduction = (1 - (mem_after / mem_before)) * 100
mem_before, mem_after, reduction


✅ **Criterio**: reducción ≥ 50%.  
Si tu reducción es menor, revisa:
- más columnas `object` que deban ser `category`
- columnas numéricas que sigan en `int64/float64`
- leer el CSV directamente con `dtype=` (optimización desde la lectura).


## 3) Fase 2: Ingeniería de Variables (vectorizada)

Se crean variables **sin `.apply()`**, usando operaciones vectorizadas:

- Variables temporales: hora, día de la semana
- Variable geoespacial: distancia Haversine entre pickup y dropoff
- Ejemplos de `.assign()` y `.query()`


In [ ]:
def haversine_km(lat1, lon1, lat2, lon2):
    """Distancia Haversine vectorizada (km)."""
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Nombres típicos del dataset
pickup_lat = "pickup_latitude"
pickup_lon = "pickup_longitude"
drop_lat = "dropoff_latitude"
drop_lon = "dropoff_longitude"

df_feat = df_opt.copy()

# Variables temporales
if "pickup_datetime" in df_feat.columns:
    df_feat = df_feat.assign(
        pickup_hour=df_feat["pickup_datetime"].dt.hour.astype("int16"),
        pickup_dow=df_feat["pickup_datetime"].dt.dayofweek.astype("int8"),  # 0=Lunes
        pickup_date=df_feat["pickup_datetime"].dt.date
    )

# Distancia (si existen columnas)
if all(c in df_feat.columns for c in [pickup_lat, pickup_lon, drop_lat, drop_lon]):
    df_feat["haversine_km"] = haversine_km(
        df_feat[pickup_lat], df_feat[pickup_lon],
        df_feat[drop_lat], df_feat[drop_lon]
    ).astype("float32")

df_feat.head()


In [ ]:
# Ejemplo de query: duración positiva y distancia razonable (ajusta si necesitas)
if "trip_duration" in df_feat.columns and "haversine_km" in df_feat.columns:
    df_clean = df_feat.query("trip_duration > 0 and haversine_km >= 0 and haversine_km < 200")
else:
    df_clean = df_feat.copy()

df_clean.shape


## 4) Fase 3: Visualización Avanzada (Storytelling)

Objetivo: generar gráficos que **cuenten una historia**:

- **Distribuciones sesgadas** → usar escala log
- **Overplotting** → transparencia (`alpha`), tamaño (`s`) o `hexbin`
- **Relaciones temporales** → heatmap día vs hora
- Limpieza de “chartjunk”: títulos claros, ejes, `despine()`, etc.


In [ ]:
ASSETS = Path("assets")
ASSETS.mkdir(exist_ok=True)


In [ ]:
# 4.1 Distribución: trip_duration (sesgada) con escala log
fig, ax = plt.subplots(figsize=(12, 5))

if "trip_duration" in df_clean.columns:
    sns.histplot(df_clean["trip_duration"], bins=100, ax=ax)
    ax.set_xscale("log")
    ax.set_title("Distribución de duración de viajes (escala log)")
    ax.set_xlabel("trip_duration (seg) [log]")
    ax.set_ylabel("Frecuencia")
else:
    ax.text(0.5, 0.5, "No existe 'trip_duration' en el dataset", ha="center", va="center")

sns.despine()
p1 = ASSETS / "hist_trip_duration_log.png"
fig.savefig(p1, dpi=300, bbox_inches="tight")
plt.close(fig)
p1


In [ ]:
# 4.2 Overplotting geoespacial: scatter con alpha y tamaño pequeño
fig, ax = plt.subplots(figsize=(7, 7))

if all(c in df_clean.columns for c in [pickup_lat, pickup_lon]):
    sample = df_clean.sample(min(100000, len(df_clean)), random_state=42)
    ax.scatter(sample[pickup_lon], sample[pickup_lat], s=1, alpha=0.05)
    ax.set_title("Pickup locations (sample) — alpha para overplotting")
    ax.set_xlabel("pickup_longitude")
    ax.set_ylabel("pickup_latitude")
else:
    ax.text(0.5, 0.5, "No existen columnas de coordenadas", ha="center", va="center")

sns.despine()
p2 = ASSETS / "pickup_scatter_alpha.png"
fig.savefig(p2, dpi=300, bbox_inches="tight")
plt.close(fig)
p2


In [ ]:
# 4.3 Alternativa a overplotting: Hexbin (densidad)
fig, ax = plt.subplots(figsize=(7, 7))

if all(c in df_clean.columns for c in [pickup_lat, pickup_lon]):
    sample = df_clean.sample(min(400000, len(df_clean)), random_state=42)
    hb = ax.hexbin(sample[pickup_lon], sample[pickup_lat], gridsize=80, mincnt=1)
    ax.set_title("Densidad de pickups (hexbin)")
    ax.set_xlabel("pickup_longitude")
    ax.set_ylabel("pickup_latitude")
    fig.colorbar(hb, ax=ax, label="conteo")
else:
    ax.text(0.5, 0.5, "No existen columnas de coordenadas", ha="center", va="center")

sns.despine()
p3 = ASSETS / "pickup_hexbin.png"
fig.savefig(p3, dpi=300, bbox_inches="tight")
plt.close(fig)
p3


In [ ]:
# 4.4 Heatmap temporal: día vs hora (requiere pickup_dow y pickup_hour)
fig, ax = plt.subplots(figsize=(12, 5))

if all(c in df_clean.columns for c in ["pickup_dow", "pickup_hour"]):
    pivot = (
        df_clean.groupby(["pickup_dow", "pickup_hour"])
        .size()
        .reset_index(name="trips")
        .pivot(index="pickup_dow", columns="pickup_hour", values="trips")
        .fillna(0)
    )
    sns.heatmap(pivot, ax=ax)
    ax.set_title("Heatmap de viajes: Día de semana (filas) vs Hora (columnas)")
    ax.set_xlabel("Hora (0-23)")
    ax.set_ylabel("Día de semana (0=Lun)")
else:
    ax.text(0.5, 0.5, "No existen pickup_dow/pickup_hour", ha="center", va="center")

sns.despine()
p4 = ASSETS / "heatmap_dow_hour.png"
fig.savefig(p4, dpi=300, bbox_inches="tight")
plt.close(fig)
p4


## 5) Fase 4: Capstone — Reporte PDF automático

Generamos un PDF que integra:
- KPIs clave (filas, memoria antes/después, % reducción)
- Visualizaciones guardadas como PNG (`assets/`)
- Narrativa breve (storytelling)


In [ ]:
OUTPUT = Path("output")
OUTPUT.mkdir(exist_ok=True)

kpi_total = len(df_raw)
kpi_reduction = float(reduction)

if "pickup_hour" in df_clean.columns:
    top_hour = int(df_clean["pickup_hour"].value_counts().idxmax())
else:
    top_hour = None

kpi_total, kpi_reduction, top_hour


In [ ]:
pdf_path = OUTPUT / "reporte_reto_modulo_1.pdf"

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Título
pdf.set_font("Arial", "B", 16)
pdf.cell(0, 10, "Reporte Automático - Reto Módulo 1", ln=True, align="C")
pdf.ln(4)

# KPIs
pdf.set_font("Arial", "", 12)
kpi_text = (
    "KPIs principales:\n"
    f"- Filas en dataset: {kpi_total:,}\n"
    f"- Memoria antes (deep): {mem_before:,.2f} MB\n"
    f"- Memoria después (deep): {mem_after:,.2f} MB\n"
    f"- Reducción: {kpi_reduction:,.2f}%\n"
    f"- Hora con más pickups (si aplica): {top_hour}\n"
)
pdf.multi_cell(0, 7, kpi_text)

# Insertar imágenes
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 8, "Visualizaciones clave", ln=True)
pdf.ln(2)

for img in [p1, p2, p3, p4]:
    if img.exists():
        pdf.image(str(img), w=180)
        pdf.ln(6)

# Narrativa final
pdf.set_font("Arial", "B", 12)
pdf.cell(0, 8, "Hallazgos (storytelling)", ln=True)
pdf.set_font("Arial", "", 12)
pdf.multi_cell(
    0, 7,
    "1) La distribución de duración de viajes es altamente sesgada; en escala log se observa mejor la concentración.\n"
    "2) En el análisis geoespacial, el overplotting oculta patrones; con alpha/hexbin aparece claramente la densidad.\n"
    "3) El patrón temporal permite identificar horas/días con mayor demanda."
)

pdf.output(str(pdf_path))
pdf_path


## 💡 Conclusiones y Hallazgos

1. **Memoria:** Se logró reducir el dataset de `X` MB a `Y` MB (deep), con una reducción de `Z%`.
2. **Patrones:** Se identificaron concentraciones geográficas y un patrón temporal de demanda.
3. **Técnica:** La vectorización evitó `apply()` y permite escalar a millones de filas con mejor rendimiento.

> Completa esta sección con tus hallazgos puntuales (máximo 5–8 líneas).
